---
# first in receipt prediction on multiple targets, fas data set 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib  
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta
os.getcwd()

In [179]:
# df = pd.read_clipboard()
# df.to_csv('./fas_057_rh.csv', index = False, encoding='utf_8_sig')
df = pd.read_csv('./fas_057_rh.csv')
df = df.fillna(method = 'ffill')
df_test = pd.read_csv('./fas_inbhdr_test.csv')
df_test = df_test.fillna(method = 'ffill')
# df_test.columns = [re.sub(r'.+\.','',i) for i in df_test.columns]
df_test = df_test[df_test['RECEIPT_DATE'] > '2021-05-10']



df_loc_inv = pd.read_csv('./location_inv.csv')
[str(datetime.strptime(i, '%Y-%m-%d %H:%M:%S.%f')-timedelta(days = 1)) for i in df_loc_inv['RECEIVED_DATE']];
df_loc_inv.head(3)
# df=df.drop(['RECEIPT_DATE','RECEIPT_DATE_TIME'], axis  = 1 )
df.head(3)


,Internal_Receipt_num,warehouse,company,receipt_id,receipt_date,CLOSE_DATE,ARRIVED_DATE_TIME,CREATION_DATE_TIME_STAMP,total_lines,TOTAL_QTY,TOTAL_CONTAINERS,TOTAL_WEIGHT
0,12347,FAS,Pernod,20170914 CHANGE CODE-3,2017-09-14 00:00:00.000,2017-09-14 09:43:55.010,2017-09-14 09:41:22.000,2017-09-14 01:40:13.000,1,150.0,1,200.0
1,12348,FAS,Solvay-SHANGHAI,CHANGEPLANT170914-1,2017-09-14 00:00:00.000,2017-09-14 10:03:17.010,2017-09-14 10:00:46.000,2017-09-14 01:51:28.000,9,12.0,9,300.0
2,12346,FAS,Solvay-SHANGHAI,CHANGEPLANT170914,2017-09-14 00:00:00.000,2017-09-14 10:03:18.533,2017-09-14 09:47:38.000,2017-09-14 01:36:58.000,10,12.0,10,300.0


In [180]:
df.columns = [i.upper() for i in df.columns.to_numpy()]
df_test.columns = [i.upper() for i in df_test.columns.to_numpy()]

def date_mutate(col, df):
    df[col] = [(datetime.strptime(i,'%Y-%m-%d %H:%M:%S.%f')+timedelta(hours = 8)) for i in df[col]]
    df[col + '_TIME'] = df[col].apply(lambda x : x.strftime('%H%M%S'))
    df[col] = df[col].apply(lambda x : x.strftime('%Y%m%d'))
    df[col+ '_2'] = df[col + '_TIME']
    # df[col + '_2'] = [datetime.strptime(i,'%H:%M:%S.%f').strftime('%H%M%S') \
    #     for i in df[col + '_TIME']]
    return df.head(1)

for i in df.columns[df.columns.str.contains(r'date|DATE')].to_numpy():
    date_mutate(i, df)

for i in df_test.columns[df_test.columns.str.contains(r'date|DATE')].to_numpy():
    date_mutate(i, df_test)



def firstdata(df):
    first_df = df.groupby(
        'CREATION_DATE_TIME_STAMP', as_index = False
        )['CREATION_DATE_TIME_STAMP'].min().reset_index()
    first_df = first_df.merge(
        df, 
        on = ['CREATION_DATE_TIME_STAMP'], 
        how = 'inner')
    return first_df

first_df = firstdata(df)
df_test_first = firstdata(df_test)

 

date_mutate('RECEIVED_DATE', df_loc_inv)


df_loc_inv_lag = (df_loc_inv[df_loc_inv['RECEIVED_DATE'] > '20170910']\
    .groupby('RECEIVED_DATE').sum().reset_index()).merge(df_loc_inv[['RECEIVED_DATE', 'RECEIVED_DATE_TIME',
       'RECEIVED_DATE_2']].groupby('RECEIVED_DATE').first(), on = 'RECEIVED_DATE', how = 'left')


In [217]:
def data_mani(first_df, df):
    train_first = first_df.groupby(
        'CREATION_DATE_TIME_STAMP'
        ).agg({'RECEIPT_ID':['nunique'], 
        'TOTAL_LINES': ['sum'], 
        'TOTAL_CONTAINERS': 'sum', 
        'TOTAL_WEIGHT':'sum',
        'TOTAL_QTY': ['sum'],
        'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()
        
    train_full = df.groupby(
        'CREATION_DATE_TIME_STAMP'
        ).agg({'RECEIPT_ID':['nunique'], 
        'TOTAL_LINES': ['sum'], 
        'TOTAL_CONTAINERS': 'sum', 
        'TOTAL_WEIGHT':'sum',
        'TOTAL_QTY': ['sum'],
        'CREATION_DATE_TIME_STAMP_2': 'max' }).reset_index()

    train_full.columns = ['CREATION_DATE_TIME_STAMP','id', 'line_sum', 'container_sum', 'weight', 'qty', 'creat_time']
    train_first.columns = ['CREATION_DATE_TIME_STAMP','id', 'line_sum', 'container_sum', 'weight', 'qty', 'creat_time']

    train_first.index= pd.to_datetime(train_first['CREATION_DATE_TIME_STAMP'], \
        yearfirst = True,  format='%Y-%m-%d')
    train_full.index = train_full['CREATION_DATE_TIME_STAMP']
    train_full.index = pd.DatetimeIndex(train_full.index)

    train_full['CREATION_DATE_TIME_STAMP'] = pd.to_datetime(train_full['CREATION_DATE_TIME_STAMP'])
    train_first['CREATION_DATE_TIME_STAMP'] = pd.to_datetime(train_first['CREATION_DATE_TIME_STAMP'])
    return train_first  , train_full 


train_first, train_full = data_mani(first_df, df)
test_first, test_full = data_mani(df_test_first, df_test)
train_first = train_first.reset_index(drop = True)
test_first = test_first.reset_index(drop = True)

In [218]:
df_loc_inv_lag['RECEIVED_DATE'] = pd.to_datetime(df_loc_inv_lag['RECEIVED_DATE'])
# df_loc_inv_lag
train_first = train_first.merge(df_loc_inv_lag, left_on='CREATION_DATE_TIME_STAMP', right_on = 'RECEIVED_DATE', how = 'left')
# sns.scatterplot(x = 'container_sum', y  ='qty', color = 'red' , data = train_full)
train_first = train_first.drop('RECEIVED_DATE', axis =  1); train_first

,CREATION_DATE_TIME_STAMP,id,line_sum,container_sum,weight,qty,creat_time,ON_HAND_QTY,TOTAL_VALUE,TOTAL_COST,TOTAL_WEIGHT,TOTAL_VOLUME,RECEIVED_DATE_TIME,RECEIVED_DATE_2
0,2017-09-11,1,2,22,22000.00,880.0,112456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-09-12,9,30,96,850242.84,512046.0,130737,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-09-13,6,9,41,41290.00,1778.0,092211,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-09-14,11,49,140,121620.00,16690.0,082402,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-09-15,4,16,36,96619.20,93797.0,094709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,2021-04-25,6,9,55,113837.57,92622.0,155955,39478.0,164995.38,1.077237e+06,315630.20,4.583909e+08,083236,083236
882,2021-04-28,1,2,3,1290.24,1008.0,133557,NaN,NaN,NaN,NaN,NaN,NaN,NaN
883,2021-04-29,1,1,10,12516.00,10080.0,131023,18.0,81.00,2.091652e+03,179.64,3.021975e+05,164536,164536
884,2021-04-30,6,7,26,116677.30,127932.0,125253,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
train_first.isnull().sum()

CREATION_DATE_TIME_STAMP      0
id                            0
line_sum                      0
container_sum                 0
weight                        0
qty                           0
creat_time                    0
ON_HAND_QTY                 411
TOTAL_VALUE                 411
TOTAL_COST                  411
TOTAL_WEIGHT                411
TOTAL_VOLUME                411
RECEIVED_DATE_TIME          411
RECEIVED_DATE_2             411
dtype: int64

In [220]:
from sklearn.impute import SimpleImputer,  KNNImputer
from sklearn.preprocessing import StandardScaler 
train_first.iloc[:, 7:] = pd.DataFrame(
    KNNImputer(n_neighbors=12, weights="uniform").fit_transform(train_first.iloc[:, 7:])
    )

In [229]:
# sns.histplot(train_full['qty'],color = 'red' )
ss = StandardScaler()
table = pd.DataFrame()
for i in train_first.columns[7:]:
    table[[i]] = ss.fit_transform(train_first[[i]]) 


In [230]:
table# 少他妈的一半数字, 不想做了.. 傻逼


,ON_HAND_QTY,TOTAL_VALUE,TOTAL_COST,TOTAL_WEIGHT,TOTAL_VOLUME,RECEIVED_DATE_TIME,RECEIVED_DATE_2
0,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
1,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
2,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
3,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
4,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
...,...,...,...,...,...,...,...
881,4.135769e+00,9.608758,7.968366e+00,3.694609,3.678614e+00,-1.346746,-1.346746
882,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000
883,-3.920354e-01,-0.372168,-4.511829e-01,-0.435116,-4.461224e-01,2.150838,2.150838
884,5.217960e-17,0.000000,-5.697863e-17,0.000000,-6.708675e-17,0.000000,0.000000


---

## deal with outbound data

In [ ]:
# ob_train = pd.read_csv('./outb_fas.csv', parse_dates=['date'])
# ob_train = ob_train.dropna(how = 'all', axis = 1)

--- 

In [ ]:
# train_first.index= pd.to_datetime(train_first['CREATION_DATE_TIME_STAMP'], yearfirst = True,  format='%Y-%m-%d')
# train_first.pop('CREATION_DATE_TIME_STAMP')
# train_full.pop('CREATION_DATE_TIME_STAMP')
# train_full['qty'] = train_full['qty'].astype(int)
# train_first['qty'] = train_first['qty'].astype(int)
# train_first['qty2'] = train_full['qty'].to_numpy() ; train_first

# train_first = train_first.reset_index()
# train_first.drop(['qty2','CREATION_DATE_TIME_STAMP','creat_time'], axis =1) 

# sktime; timeseries only prediction on trainfull


In [ ]:
# train_full.index = train_full['CREATION_DATE_TIME_STAMP']
# train_full.index = pd.DatetimeIndex(train_full.index);train_full.index
train_full_sktime = train_full.asfreq('D')
train_full_sktime = train_full_sktime.fillna(0);train_full_sktime


In [ ]:
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.regression import ComposableTimeSeriesForestRegressor
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_percentage_error

x = train_full_sktime.drop(['qty', 'CREATION_DATE_TIME_STAMP', 'creat_time'], axis = 1)
y = train_full_sktime.loc[:,'qty']
x_train, x_test , y_train, y_test = temporal_train_test_split(x, y)
#
#  # fh = ForecastingHorizon(x_test, is_relative=False)
# forecaster = ComposableTimeSeriesForestRegressor(x = x_train, y = y_train)
# forecaster.fit(x_train ,y = y_train)
# y_pred = forecaster.predict(x_test)
# mean_absolute_percentage_error(y_test, y_pred)

forecaster = NaiveForecaster(sp = 21)
forecaster.fit(y_train)

fh = ForecastingHorizon(y_test.index, is_relative=False)
y_pred = forecaster.predict(fh )
mean_absolute_percentage_error(y_test, y_pred)



In [ ]:
from sktime.regression.interval_based import  TimeSeriesForestRegressor

In [ ]:
x_train.shape[1]

In [ ]:
# forecaster = TimeSeriesForestRegressor( )
# # fh = ForecastingHorizon(x_test, is_relative=False)
# forecaster.fit(X = x_train,  y = y_train)
# y_pred = forecaster.predict(x_test)
# mean_absolute_percentage_error(y_test, y_pred)

---

# prophet prepare

In [ ]:
# train_full = train_full_sktime
train_full = train_full.asfreq('D')
train_full = train_full.fillna(0) 
train_first = train_first.asfreq('D')
train_first = train_first.fillna(0)
train_full['CREATION_DATE_TIME_STAMP'] = train_full.index
train_first['CREATION_DATE_TIME_STAMP'] = train_first.index


# train_full
train_full = train_full.reset_index(drop =True)
train_first = train_first.reset_index(drop =True)

train_fin = train_full.merge(train_first, 
    on = 'CREATION_DATE_TIME_STAMP', 
    how = 'outer',
    suffixes = ['_full', '_first'])
train_fin = train_fin.fillna(0);train_fin

In [ ]:
# train_full = train_full_sktime
test_full  = test_full.asfreq('D')
test_full  = test_full.fillna(0) 
test_first = test_first.asfreq('D')
test_first = test_first.fillna(0)
test_full['CREATION_DATE_TIME_STAMP']  = test_full.index
test_first['CREATION_DATE_TIME_STAMP'] = test_first.index


# train_full
test_full  = test_full.reset_index(drop =True)
test_first = test_first.reset_index(drop =True)

test_fin = test_full.merge(test_first, 
    on    = 'CREATION_DATE_TIME_STAMP', 
    how   = 'outer',
    suffixes = ['_full', '_first'])

test_fin = test_fin.fillna(0);test_fin

In [ ]:
def span_mani(train_fin, train_first): 
    # train_full['creat_time_first'] = ''
    train_fin['span_first'] = train_fin[
        'creat_time_first'
        ].str.slice(1,2).fillna(
            0
            ).astype(int, errors = 'ignore') + train_fin['creat_time_first'].str.slice(2,4).fillna(
                0
                ).astype(int, errors = 'ignore')/60*1 + 8

    # # train_full['creat_time_x']
    # train_first['span_full'] = train_first['creat_time_full'].str.slice(1,2).astype(int) + 
    #     train_first['creat_time_full'].str.slice(2,4).astype(int)/60*1 + 8
    train_first['span_first'] = train_first[
        'creat_time'
        ].str.slice(1,2).fillna(
            0
            ).astype(int, errors = 'ignore') + train_first['creat_time'].str.slice(2,4).fillna(
                0
                ).astype(int, errors = 'ignore')/60*1 + 8
    train_first = train_first.drop('creat_time', axis = 1)
    return train_fin, train_first

In [ ]:
train_fin, train_first = span_mani(train_fin, train_first)
test_fin, test_first   = span_mani(test_fin, test_first) 

In [ ]:
# modeles

In [ ]:
# from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.model_selection import train_test_split

x = train_first 
x['y'] = train_fin['qty_full']
# x['CREATION_DATE_TIME_STAMP'] = x.index
x_train, x_test = train_test_split(x, train_size = .75, shuffle = False, random_state = 52943)

cols = [i for i in x.columns if i not in ['y', 'CREATION_DATE_TIME_STAMP']]


In [ ]:
%%time

from fbprophet import Prophet
# df = x[['CREATION_DATE_TIME_STAMP', 'qty']]
df_naiv = x_train 
df_naiv = df_naiv.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
m = Prophet()
[m.add_regressor(i) for i in cols]

m.fit(df_naiv)

from sklearn.metrics import mean_absolute_percentage_error
df_naiv_test = x_test
df_naiv_test = df_naiv_test.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
# m.predict(x_test)
mean_absolute_percentage_error(df_naiv_test['y'], m.predict(df_naiv_test)['yhat'])
# fig = m.plot_components(m.predict(x_test))

In [ ]:
# m.plot_components(m.predict(df2))
# plt.show()

In [ ]:
# m.plot(m.predict(df2))
# plt.show()

---

 ============================== 
# pseudo labeling
 ============================== 

In [ ]:
def labeling_mani(train_first, train_full):
    train_first['dif'] = train_first['qty'] - train_full['qty'] != 0
    train_first['dif'] = [1 if i == True else 0 for i in train_first['dif']]

    train_full['dif'] = train_first['qty'] - train_full['qty'] != 0
    train_full['dif'] = [1 if i == True else 0 for i in train_full['dif']]
    return train_first , train_full
# train_full['diff'] = train_full['qty_x'] - train_full['qty_y'] ==  0
# train_full['diff'] = [1 if i == True else 0 for i in train_full['diff']]
# train_full = train_full.asfreq('D')

In [ ]:
labeling_mani(train_first, train_full)
test_first, test_full = labeling_mani(test_first, test_full)

In [ ]:
x = train_first.drop('y', axis  =1 )
x_train, x_test = train_test_split(x, train_size = .75, shuffle = False, random_state = -1)


In [ ]:
%%time
from fbprophet import Prophet 
df_lb = x_train 

## qty_x s是最终预测值, 所以要drop
cols_lb = [i for i in df_lb.columns 
    if i not in ['dif', 'CREATION_DATE_TIME_STAMP']]

# df_lb = df_lb.drop(['creat_time_x','creat_time_y', 'qty_x'], axis =1 )
df_lb = df_lb.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'dif': 'y'}, axis = 1)
m = Prophet(daily_seasonality=True) 

"""
0.12379144878303337 daily_seasonality=True; empty 0.12318682390845224
"""


[m.add_regressor(i) for i in cols_lb]

m.fit(df_lb)

# ?mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import  mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import mean_absolute_error
from sklearn.metrics import accuracy_score

# x_test1 = x_test.drop(['dif'], axis =1 )
x_test1 = x_test.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'dif': 'y'}, axis = 1)
 
# m.predict(x_test)
forecast = m.predict(x_test1)
mean_absolute_error(x_test1['y'], m.predict(x_test1)['yhat'])
# fig = m.plot_components(m.predict(x_test))

In [ ]:
# # %%warning = False
# from prophet.plot import plot_plotly, plot_components_plotly
# plot_plotly(m, forecast)
 

In [ ]:
lb_prophet_result = pd.DataFrame(forecast['yhat'])
lb_prophet_result['abs_diff'] = np.abs(forecast['yhat'] - .5)
lb_prophet_result.sort_values('abs_diff').iloc[0,0]

In [ ]:

acc_score_data  = [1 if i>=lb_prophet_result['yhat'].mean() \
    else 0 for i in forecast['yhat']]

acc_score_data1 = [1 if i>=lb_prophet_result.sort_values('abs_diff').iloc[0,0] \
    else 0 for i in forecast['yhat']]
    
acc_score_data2 = [1 if i>lb_prophet_result['yhat'].std() \
    else 0 for i in forecast['yhat']]

# mean_absolute_error(x_test1['y'], m.predict(x_test1)['yhat'])
[accuracy_score(x_test1['y'],i) for i in [acc_score_data, acc_score_data1, acc_score_data2]]


 ============================== 
## pred 2 the id number predicting:
 ============================== 

In [ ]:
# pred 2 the id number predicting:
id_cnt_train = train_first.copy()
id_cnt_train = id_cnt_train[id_cnt_train['dif'] != 0]
id_cnt_train['y'] = train_full['id']
"""
[4.806768544455207, 1.1680515078598923]
[7.133413104312447, 0.6253522370101863]  # id
"""
# id_cnt_train['y'] = train_full['line_sum']

"""
[16.60773539220494, 0.6975939132491072] # line_sum
"""
id_cnt_train

In [ ]:
# x = id_cnt_train.drop('y', axis  =1 )
x_train, x_test = train_test_split(id_cnt_train,\
     train_size = .75, shuffle = False, random_state = None)

## qty_x s是最终预测值, 所以要drop
cols_lb = [i for i in x_train.columns 
    if i not in ['y', 'CREATION_DATE_TIME_STAMP']]

x_train = x_train.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
m2 = Prophet(daily_seasonality=True) 

[m2.add_regressor(i) for i in cols_lb]

m2.fit(x_train)

In [ ]:

# x_test1 = x_test.drop(['dif'], axis =1 )
x_test1 = x_test.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
 
# m.predict(x_test)
forecast_2 = m2.predict(x_test1)
[mean_absolute_error(x_test1['y'], forecast_2['yhat']),\
     mean_absolute_percentage_error(x_test1['y'], forecast_2['yhat'])]

 ============================== 
# test data validate 
 
 
 ============================== 


In [ ]:
train_all = x
train_all = train_all.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'dif': 'y'}, axis = 1)
# m2 = Prophet(daily_seasonality=True) 
# m2.fit(train_all)

In [ ]:
# a
test_all  = test_first.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'dif': 'y'}, axis = 1)
test_pred = m.predict(test_all)
 
mean_absolute_error(test_all['y'], test_pred['yhat'])

In [ ]:
acc_tst = [1 if i>=test_pred['yhat'].mean() \
    else 0 for i in test_pred['yhat']]
accuracy_score(acc_tst, test_all['y'])

In [ ]:
import nbformat
nbformat.__version__

In [ ]:
import plotly.express as px

In [ ]:
# %%warning = False
test_pre_p = go.Figure()
test_pre_p.add_trace(go.Scatter(x = test_pred['ds'],\
    y = test_pred['yhat'], 
    mode = 'markers',
    name = 'pred'))
test_pre_p.add_trace(go.Scatter(x = test_all['ds'], \
    y = test_all['y'], 
    mode = 'markers',
    name = 'original_out'))


In [ ]:
# # %%warning = False
# test_pre_p = go.Figure()
# test_pre_p.add_trace(go.Scatter(x = test_all['ds'], \
#     y = test_all['y'], 
#     mode = 'markers',
#     name = 'original_out'))
# test_pre_p.add_trace(go.Scatter(x = test_pred['ds'],\
#     y = test_pred['yhat'], 
#     mode = 'markers',
#     name = 'pred'))
# test_pre_p.show() 

----
## try h2o predict qty

In [ ]:
x_train2 = x_train[x_train['dif'] == 1] 
x_test2 = x_test[x_test['dif'] == 1];len(x_test2)
x_train2['qty_tar'] = train_full[train_full['dif'] == 1].iloc[0:len(x_train2), :]['qty']
x_test2['qty_tar'] = train_full[train_full['dif'] == 1].iloc[len(x_train2):, :]['qty'];x_train2

In [ ]:
import h2o
from h2o.estimators import H2OIsolationForestEstimator
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators import H2OXGBoostEstimator
h2o.init()


In [ ]:
cols_h2o = [i for i in x_train2.columns.to_numpy() if i not in 'qty_tar']

# Import the prostate dataset
h2o_df = h2o.H2OFrame(x_train2)
# Split the data giving the training dataset 75% of the data
train,test = h2o_df.split_frame(ratios=[0.75])

# Build an Isolation forest model
model = H2OIsolationForestEstimator(sample_rate = 0.1,
                                    max_depth = 20,
                                    ntrees = 500)
model.train(training_frame=train,x =cols_h2o)

# Calculate score
score = model.predict(test)
result_pred = score["predict"]

# Predict the leaf node assignment
ln_pred = model.predict_leaf_node_assignment(test, "Path")

In [ ]:
score_train = model.predict(train)
forest_pred_trai = pd.concat([score_train['predict'].as_data_frame(),\
    result_pred.as_data_frame() ], axis = 0).reset_index()
forest_pred_test = model.predict(h2o.H2OFrame(x_test2))

In [ ]:
train.as_data_frame()
train.as_data_frame().append(score_train.as_data_frame())

In [ ]:
score2

In [ ]:
# Generate predictions on a test set (if necessary):
pred = dl.predict(test)

In [ ]:
sult = pd.concat([test.as_data_frame()['qty_tar'], pred.as_data_frame()], axis = 1)

In [ ]:
sult

In [ ]:
mean_absolute_percentage_error(sult['qty_tar'], sult['predict'])

In [ ]:
x_train2 = x_train2.reset_index()
x_test2 = x_test2.reset_index()
x_train2['labels'] = forest_pred_trai['predict'] 
x_test2['labels'] = forest_pred_test.as_data_frame()['predict']

In [ ]:
predictors = [i for i in x_train2.columns.to_numpy() if i not in 'qty_tar']
response = 'qty_tar'
# Import the prostate dataset
h2o_df = h2o.H2OFrame(x_train2)
# Split the data giving the training dataset 75% of the data
train,test = h2o_df.split_frame(ratios=[0.75])

dl = H2ODeepLearningEstimator(distribution="AUTO",
                               hidden=[300,300],
                               epochs=600,
                               train_samples_per_iteration=-1,
                               reproducible=True,
                               activation="Rectifierwithdropout",
                               single_node_mode=False,
                               balance_classes=False,
                               force_load_balance=False,
                               seed=707,
                               tweedie_power=1.5,
                               score_training_samples=0,
                               score_validation_samples=0,
                               score_each_iteration = True,
                               stopping_rounds=0)
dl.train(x=predictors,
         y=response,
          training_frame= train,
          validation_frame = test)

# Eval performance:
perf = dl.model_performance();perf


In [ ]:
perf

In [ ]:
test_lb_qty_test2 = dl.predict(test).as_data_frame()

In [ ]:
test_lb_qty_test2

In [ ]:
mean_absolute_percentage_error(sult['qty_tar'], sult['predict'])

----
## Darts

In [ ]:
df_lb , x_test1 

"""
the dataframes are taken from fbprophet model, 
no need to split anyway.
"""

In [ ]:
from darts import TimeSeries

In [ ]:
from darts import TimeSeries

series_train = TimeSeries.from_dataframe(df_lb, 
       time_col = 'ds', 
       value_cols ='y')
series_test = TimeSeries.from_dataframe(x_test1, 
       time_col = 'ds', 
       value_cols =  'y')
series_train_cov = TimeSeries.from_dataframe(df_lb,\
       'ds',
       ['id', 'line_sum', 'container_sum', 'weight', 'qty', 'span_first'])
series_test_cov = TimeSeries.from_dataframe(x_test1, 
       'ds', 
       ['id', 'line_sum', 'container_sum', 'weight', 'qty', 'span_first'] )

In [ ]:
ful_span = TimeSeries.from_dataframe(df_lb.append(x_test1), \
    'ds', 
    value_cols = 'y')
ful_span_cov = TimeSeries.from_dataframe(df_lb.append(x_test1), \
    'ds', 
    ['id', 'line_sum', 'container_sum', 'weight', 'qty', 'span_first'] )

In [ ]:
from darts.models import RNNModel
md1_bert = NBEATSModel(input_chunk_length=31, \
    output_chunk_length=15, n_epochs=5, random_state=43)
    
# md2_lstm = BlockRNNModel(model='LSTM', \
#     input_chunk_length=12, output_chunk_length=6, n_epochs=5, random_state=43)


 > ## *Past Covariates denote time series whose past values are known at prediction time. These are usually things that have to be measured or observed. * Future Covariates denote time series whose future values are already known at prediction time for the span of the forecast horizon. These can for instance represent known future holidays, or weather forecasts.*
> 
> 
 > ## input_chunk_length: this is the length of the lookback window of the model; so each output will be computed by the model by reading the previous input_chunk_length points. 

In [ ]:
%%time
md1_bert.fit(series = series_train, \
    past_covariates = series_train_cov,
    verbose=True)

In [ ]:
%%time
backtest_bert = md1_bert.historical_forecasts(series_train,\
    past_covariates = series_train_cov,
    start = .75,
    retrain = False)

# backtest_cov = model_cov.historical_forecasts(series_air_scaled,
#                                               past_covariates=air_covariates,
#                                               start=0.6,
#                                               forecast_horizon=12,
#                                               stride=1,
#                                               retrain=False,
#                                               verbose=True)


In [ ]:
backtest_bert.values()

In [ ]:
%%time
train_pred_bert = md1_bert.predict(n = 334 , 
    past_covariates = series_train_cov,
    # future_covariates = series_test_cov
    )
train_pred_bert.values()

In [ ]:
from darts.metrics import mape
mean_absolute_error(train_pred_bert.values(), series_test.values())

## lSTM in darts

In [ ]:
%%time
LSTM_darts = RNNModel(
    model='LSTM',
    hidden_dim=20,
    dropout=0,
    batch_size=7,
    n_epochs=5,
    optimizer_kwargs={'lr': 1e-3},
    model_name='lstm_RNN',
    log_tensorboard=True,
    random_state=42,
    # training_length=20,
    input_chunk_length=31,
    force_reset=True
)
LSTM_darts.fit(series_train,
             future_covariates=series_train_cov,
            #  val_series=series_test,
            #  val_future_covariates=series_test_cov,
             verbose=True)

In [ ]:
np.set_printoptions(suppress=True)
lstm_pred_test = LSTM_darts.predict(n = 334, future_covariates = ful_span_cov).values()

In [ ]:
acc_score_data  = [1 if i>=lstm_pred_test.mean() \
    else 0 for i in lstm_pred_test]

# acc_score_data1 = [1 if i>=lstm_pred_test.sort_values() \
#     else 0 for i in forecast['yhat']]
    
# acc_score_data2 = [1 if i>lb_prophet_result['yhat'].std() \
#     else 0 for i in forecast['yhat']]

# mean_absolute_error(x_test1['y'], m.predict(x_test1)['yhat'])
# [accuracy_score(x_test1['y'],i) for i in [acc_score_data, acc_score_data1, acc_score_data2]]
accuracy_score(x_test1['y'],acc_score_data) 

In [ ]:
lstm_plt = x_test1.copy()
lstm_plt['y_pred'] = lstm_pred_test

In [ ]:
import plotly.express as ex
fig = ex.scatter(data_frame=lstm_plt, x = 'ds', y = 'y')
fig.add_scatter(x = lstm_plt['ds'],  y = lstm_plt['y_pred'])
fig.show()

In [ ]:
# import altair as alt
# brush = alt.selection_interval()
# alt.Chart(lstm_plt).mark_point().encode(
#     x='ds',
#     y='y'
# ).add_selection(
#     brush
# )
 

In [ ]:
# min(abs(lstm_plt['y_pred'] - .5))

lstm_plt['abs_diff'] = np.abs(lstm_plt['y_pred'] - .5)
lstm_plt.sort_values('abs_diff')

In [ ]:
pred_1_lable1 = [1 if i> 0.565032 else 0 for i in lstm_plt['y_pred']]
# lstm_plt['y_pred']
pred_1_lable2 = [1 if i> lstm_plt['y_pred'].mean() else 0 for i in lstm_plt['y_pred']]

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
[accuracy_score(lstm_plt['y'], pred_1_lable1),
accuracy_score(lstm_plt['y'], pred_1_lable2)]

In [ ]:


# series.plot(label='data')
# for i, m in enumerate(berkets):
#     err = mape(backtests[i], series)
#     backtests[i].plot(lw=3, label='{}, MAPE={:.2f}%'.format(m, err))

# plt.title('Backtests with 3-months forecast horizon')
# plt.legend()

---

## qty prediction, filter data which not 1

In [ ]:
x_train2 = x_train[x_train['dif'] == 1] 
x_test2 = x_test[x_test['dif'] == 1];len(x_test2)
x_train2['qty_tar'] = train_full[train_full['dif'] == 1].iloc[0:len(x_train2), :]['qty']
x_test2['qty_tar'] = train_full[train_full['dif'] == 1].iloc[len(x_train2):, :]['qty']

In [ ]:
[len(x_train2),len(x_test2)]

In [ ]:
df_2_all = pd.DataFrame(pd.concat([x_train2, x_test2], axis = 0)).reset_index(); df_2_all

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

tran3 = ColumnTransformer([
        ('somename', StandardScaler(), ['weight', 'qty', 'qty_tar'])
    ])
tran3.fit(df_2_all[['weight', 'qty', 'qty_tar']])

df_2_all[['weight', 'qty',  'qty_tar']] = pd.DataFrame(
    tran3.fit_transform(df_2_all[['weight', 'qty', 'qty_tar']])

) 

In [ ]:
# tran2.fit(x_train2[['weight', 'qty', 'span_first']])
# tran2.transform(x_test2[['weight', 'qty', 'span_first']])

df_2_all =df_2_all.fillna(0)
x_train2 = df_2_all.iloc[0:652,:]
x_test2 = df_2_all.iloc[652:,:]

In [ ]:
from fbprophet import Prophet 
df_lb = x_train2

## qty_x s是最终预测值, 所以要drop
cols_lb = [i for i in df_lb.columns 
    if i not in ['qty_tar', 'CREATION_DATE_TIME_STAMP']]

# df_lb = df_lb.drop(['creat_time_x','creat_time_y', 'qty_x'], axis =1 )
df_lb = df_lb.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'qty_tar': 'y'}, axis = 1)
m = Prophet(daily_seasonality=True)
[m.add_regressor(i) for i in cols_lb]

m.fit(df_lb)


# ?mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import  mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import mean_absolute_error
from sklearn.metrics import accuracy_score

# x_test1 = x_test.drop(['dif'], axis =1 )
x_test1 = x_test2.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'qty_tar': 'y'}, axis = 1)
 
# m.predict(x_test)
forecast = m.predict(x_test1)
mean_absolute_percentage_error(x_test1['y'], m.predict(x_test1)['yhat'])
# fig = m.plot_components(m.predict(x_test))

---
# cross validation on prophet

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
df_cv = cross_validation(m, initial='365 days', period='31 days', horizon = '180 days')


In [ ]:
df_cv = pd.DataFrame(df_cv);df_cv

In [ ]:
sns.scatterplot(data = df_cv, x = 'ds', y = 'yhat')

In [ ]:
performance_metrics(df_cv).sort_values('mae')

## Tunning paras

In [ ]:
from fbprophet import Prophet 
df_lb = x_train 
cols_lb = [i for i in df_lb.columns 
    if i not in ['diff','qty_x','creat_time_x','creat_time_y','CREATION_DATE_TIME_STAMP']]

df_lb = df_lb.drop(['creat_time_x','creat_time_y', 'qty_x'], axis =1 )
df_lb = df_lb.rename({'CREATION_DATE_TIME_STAMP' : 'ds', 'diff': 'y'}, axis = 1)
m = Prophet()
[m.add_regressor(i) for i in cols_lb]

m.fit(df_lb)

In [ ]:
%%Time
import itertools
param_grid = {  
    'changepoint_prior_scale': [0.01,  0.5],
    'seasonality_prior_scale': [0.01, 0.1],
}
 

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params)  # Fit model with given params
    [m.add_regressor(i) for i in cols_lb]
    m.fit(df_lb)
    df_cv = cross_validation(m, 
        initial='365 days', period='31 days', horizon = '180 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
# Python
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
df_lb

In [ ]:
x_test1 = x_test.drop(['creat_time_x','creat_time_y', 'qty_x', 'diff'], axis =1 )
x_test1 = x_test1.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
x_test1['y'] = 0
m.predict(x_test1)
mean_absolute_error(x_test['diff'], m.predict(x_test1)['yhat'])

In [ ]:
values = m.predict(x_test1)['yhat']
labels = [1 if i>.5 else 0 for i in values]

In [ ]:
x_test['pred_labels'] = labels;x_test


In [ ]:
accuracy_score(x_test['diff'], x_test['pred_labels'])

In [ ]:
x_train

In [ ]:
# qty_x is target
from fbprophet import Prophet 
df_value = x_train 
df_value = df_value.drop(['creat_time_x','creat_time_y'], axis =1 )

df_value = df_value.rename({
    'CREATION_DATE_TIME_STAMP' : 'ds', 'qty_x': 'y', 'diff':'pred_labels'
    }, axis = 1)

    
cols_value = [i for i in df_value.columns 
    if i not in ['y', 'ds', 'pred_labels']]

m = Prophet()
[m.add_regressor(i) for i in cols_value]

m.fit(df_value)

In [ ]:
df_value

In [ ]:
x_test_fin = x_test.drop(['creat_time_x','creat_time_y', 'qty_x', 'diff'], axis =1 )
 
x_test_fin = x_test_fin.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1);
x_test_fin

In [ ]:

# m.predict(x_test_fin)
mean_absolute_percentage_error(x_test['qty_x'], m.predict(x_test_fin)['yhat'])

# aaa

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(
    n_splits = 6, 
    test_size = .25, 
    # max_train_size = .75
    )
alg1 = Lasso()

cv_results_1 = model_selection.cross_validate(
    alg1, 
    train_first.drop(['qty2','CREATION_DATE_TIME_STAMP','creat_time'], axis =1) , train_first['qty2'].to_numpy(), 
    cv  = cv_split, scoring='neg_mean_absolute_percentage_error')


cv_results_1['test_score'].mean()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
alg2 = RandomForestRegressor(n_estimators = 34, 
        max_depth = 61,
        min_samples_split = 3, 
        min_samples_leaf=4, 
        # max_leaf_nodes = 31
        )
cv_results_2 = model_selection.cross_validate(
        alg2, 
        train_first, 
        train_full['qty'], 
        cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

In [ ]:
alg2.fit(train_first,train_full['qty']).predict(train_first)

In [ ]:
from sklearn.neural_network import MLPRegressor
alg3 = MLPRegressor(
    hidden_layer_sizes = (891, 891, 450, 135), 
    max_iter=1000,
    verbose=True, 
    activation = 'identity', 
    solver = 'adam', learning_rate = 'adaptive')
cv_results_3 = model_selection.cross_validate(
    alg3, train_first,train_full['qty'], 
    cv  = cv_split, 
    scoring='neg_mean_absolute_percentage_error')
cv_results_3['test_score'].mean()

In [ ]:
from sklearn.neural_network import MLPRegressor
alg3 = MLPRegressor(
    hidden_layer_sizes = (1120, 850, 450, 135), 
    max_iter=1000,
    verbose=True, 
    activation = 'relu', 
    solver = 'adam', 
    learning_rate = 'adaptive',
    warm_start = True,
    early_stopping = True
    )
cv_results_3 = model_selection.cross_validate(
    alg3, train_first,train_full['qty'], 
    cv  = cv_split, 
    scoring='neg_mean_absolute_percentage_error')
cv_results_3['test_score'].mean()